In [53]:
%load_ext autoreload
%autoreload 2

from configs import project_config
import pickle
import numpy as np
import torch
import os
from src.generative_modelling.models.ClassVPSDEDiffusion import VPSDEDiffusion
from src.generative_modelling.models.TimeDependentScoreNetworks.ClassConditionalLSTMTSPostMeanScoreMatching  import \
    ConditionalLSTMTSPostMeanScoreMatching

import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
from src.classes.ClassFractionalLorenz96 import FractionalLorenz96
from configs.RecursiveVPSDE.LSTM_4DLorenz.recursive_LSTM_PostMeanScore_4DLorenz_T256_H05_tl_110data import get_config
config = get_config()
num_time_steps = 10
num_paths = 3
fLnz = FractionalLorenz96(X0=np.array(config.initState), diff=config.diffusion, forcing_const=config.forcing_const,
                              num_dims=config.ndims)
Xs = np.array(
        [fLnz.euler_simulation(H=config.hurst, N=num_time_steps, deltaT=config.deltaT, X0=np.array(config.initState), Ms=None, gaussRvs=None,
                               t0=config.t0, t1=config.t0*config.deltaT) for _ in (range(num_paths))]).reshape((num_paths, num_time_steps+1, config.ndims))
Xs = Xs[:, np.random.choice(np.arange(Xs.shape[1]),size=1)[0], :]
print(Xs.shape)

(3, 4)


In [63]:
from torch.nn.utils.rnn import pad_sequence
from configs.RecursiveVPSDE.LSTM_4DLorenz.recursive_LSTM_PostMeanScore_4DLorenz_T256_H05_tl_110data import get_config
config = get_config()
sim_data = np.load(config.data_path, allow_pickle=True)
sim_data_tensor = torch.tensor(sim_data, dtype=torch.float)
#dX_dims = [np.diff(Xs_i)[0] / 250 for Xs_i in [Xs0, Xs1, ..., Xs_{D-1}]]
device = "cpu"
PM = ConditionalLSTMTSPostMeanScoreMatching(*config.model_parameters).to(device)
dX_global = 1./1000
def process_single_threshold(x):
    diff = sim_data_tensor - x.reshape(1,-1)  # shape: (M, N, D)
    mask = torch.all((diff <= dX_global) & (diff >= -dX_global), dim=-1)
    # Get indices where mask is True (each index is [i, j])
    indices = mask.nonzero(as_tuple=False)
    sequences = []
    js = []
    for idx in indices:
        i, j = idx.tolist()
        # Extract the sequence: row i, columns 0 to j (inclusive)
        seq = sim_data_tensor[i, :j + 1, :]
        sequences.append(seq)
        js.append(len(seq))
    outputs = []
    if sequences:
        # Pad sequences to create a batch.
        # pad_sequence returns tensor of shape (batch_size, max_seq_len)
        padded_batch = pad_sequence(sequences, batch_first=True, padding_value=torch.nan).to(device)
        # Add feature dimension: now shape becomes (batch_size, max_seq_len, 1)
        #padded_batch = padded_batch.unsqueeze(-1).to(device)
        with torch.no_grad():
            batch_output, _ = PM.rnn(padded_batch, None)
        outputs = batch_output[torch.arange(batch_output.shape[0]), torch.tensor(js, dtype=torch.long) - 1,
                  :].unsqueeze(1).cpu()
    return x, outputs

# Option 1: Process sequentially (using tqdm)
features_Xs = {}
for i in range(Xs.shape[0]):
    x = Xs[i, :].reshape(-1, 1)
    x_val, out = process_single_threshold(x)
    assert (len(out) > 0)
    features_Xs[tuple(x_val.squeeze().tolist())] = out

In [ ]:
from src.classes.ClassFractionalBiPotential import FractionalBiPotential
from configs.RecursiveVPSDE.LSTM_fQuadSinHF.recursive_LSTM_PostMeanScore_fQuadSinHF_T256_H05_tl_110data import get_config as get_config
from src.classes.ClassFractionalQuadSin import FractionalQuadSin
from configs.RecursiveVPSDE.LSTM_fBiPot.recursive_LSTM_PostMeanScore_fBiPot_T256_H05_tl_110data_SbleTgt import get_config as get_config
config = get_config()
print(config.data_path)
print(config.scoreNet_trained_path)
num_paths = 0
num_time_steps = config.ts_length
isUnitInterval = True
diff = 1
initial_state = 0.
rvs = None
H = 0.5
deltaT = config.deltaT
t0 = config.t0
t1 = config.t1
if "QuadSin" in config.data_path:
    fBiPot = FractionalQuadSin(quad_coeff=config.quad_coeff, sin_space_scale=config.sin_space_scale, sin_coeff = config.sin_coeff, diff=diff, X0=initial_state)
elif "BiPot" in config.data_path:
    fBiPot = FractionalBiPotential(quartic_coeff=config.quartic_coeff, quad_coeff=config.quad_coeff, const = config.const, diff=diff, X0=initial_state)
if num_paths > 0:
    sim_data = np.array(
        [fBiPot.euler_simulation(H=H, N=num_time_steps, deltaT=deltaT, isUnitInterval=isUnitInterval, X0=initial_state, Ms=None, gaussRvs=rvs,
                               t0=t0, t1=t1) for _ in (range(num_paths))]).reshape((num_paths, num_time_steps+1))

    plt.hist(sim_data.flatten(), density=True, bins=150)
    print(max(np.abs(sim_data.flatten())))
    plt.show()
    plt.close()
diffs = []
paths = []
for k in range(num_paths):
    path = sim_data[k,:]
    if "QuadSin" in config.data_path:
        drifts = -2*config.quad_coeff * path[:-1] + config.sin_space_scale*config.sin_coeff*np.sin(config.sin_space_scale*path[:-1])
    elif "BiPot" in config.data_path:
        drifts = -(4*config.quartic_coeff * np.power(path[:-1],3) + config.quad_coeff*path[:-1] + config.const)
    #plt.scatter(path[:-1], drifts[1:]*deltaT, s=1, label="True")
    d = (np.diff(path)[np.abs(path[1:])<np.inf])
    if len(d) > 0:
        diffs.append(d)
    paths.append(path)
if num_paths > 0 and ("QuadSin" in config.data_path):
    cs, bins, _ = plt.hist(config.sin_space_scale*np.concatenate(diffs).flatten(), bins=100, density=True, label="Histogram")
    print(max(config.sin_space_scale*np.abs(np.concatenate(diffs).flatten())))
    plt.vlines(x=1., ymin=0., ymax=max(cs), color="orange", label="Frequency Limit")
    plt.legend()
    plt.show()
    plt.close()


In [ ]:
def rmse_ignore_nans(y_true, y_pred):
    assert (y_true.shape[0] == y_pred.shape[0])
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    mask = ~np.isnan(y_true) & ~np.isnan(y_pred)  # Ignore NaNs in both arrays
    return np.sqrt(np.mean((y_true[mask] - y_pred[mask]) ** 2))

In [ ]:
from src.generative_modelling.models.TimeDependentScoreNetworks.ClassConditionalLSTMTSScoreMatching import ConditionalLSTMTSScoreMatching

config = get_config()
print(config.beta_min)
if config.has_cuda:
    device = int(os.environ["LOCAL_RANK"])
else:
    print("Using CPU\n")
    device = torch.device("cpu")

diffusion = VPSDEDiffusion(beta_max=config.beta_max, beta_min=config.beta_min)

max_diff_steps = config.max_diff_steps
sample_eps = config.sample_eps
ts_step = 1 / config.ts_length

Nepoch = 1440
ts_type_num = 1
toSave = False
# Fix the number of training epochs and training loss objective loss
if "PM" in config.scoreNet_trained_path:
    PM = ConditionalLSTMTSPostMeanScoreMatching(*config.model_parameters).to(device)
else:
    PM = ConditionalLSTMTSScoreMatching(*config.model_parameters).to(device)
PM.load_state_dict(torch.load(config.scoreNet_trained_path + "_NEp" + str(Nepoch)))

In [ ]:
%load_ext autoreload
%autoreload 2
from configs.RecursiveVPSDE.LSTM_fQuadSinHF.recursive_LSTM_PostMeanScore_fQuadSinHF_T256_H05_tl_110data import get_config
gconfig = get_config()
print(gconfig.loss_factor)
with open(gconfig.scoreNet_trained_path.replace("/trained_models/", "/training_losses/") + "_loss", 'rb') as f:
        losses = np.array(pickle.load(f))
start_idx = 0
plt.scatter(np.arange(start_idx, losses.shape[0]), losses[start_idx:],s=2, label="Small")
#plt.hlines(np.sqrt(config.deltaT), xmin = start_idx, xmax=losses.shape[0]-start_idx, color="red")
plt.yscale("log")
plt.legend()
losses[-1]/losses[start_idx]

In [ ]:
from configs.RecursiveVPSDE.LSTM_fBiPot.recursive_LSTM_PostMeanScore_fBiPot_T256_H05_tl_110data_SbleTgt import get_config as get_config
gconfig2 = get_config()
with open(gconfig2.scoreNet_trained_path.replace("/trained_models/", "/training_losses/") + "_loss", 'rb') as f:
        print(f)
        losses2 = np.array(pickle.load(f))[:losses.shape[0]]
plt.scatter(np.arange(start_idx, losses2.shape[0]), losses2[start_idx:],s=2, label="Stable Target")
#plt.hlines(np.sqrt(config.deltaT), xmin = start_idx, xmax=losses.shape[0]-start_idx, color="red")
plt.yscale("log")
plt.legend()
losses2[-1]/losses2[start_idx]

In [ ]:
start_idx = 00
def compute_ema(loss_tensor, beta):
    ema_values = np.zeros_like(loss_tensor)  # Initialize EMA tensor
    ema_values[0] = loss_tensor[0]  # First value stays the same

    for i in range(1, len(loss_tensor)):
        ema_values[i] = beta * ema_values[i - 1] + (1 - beta) * loss_tensor[i]

    return ema_values
# Define EMA decay rates
beta_short = 0.9   # Short-term trend (reacts quickly)
beta_long = 0.99   # Long-term trend (smoother)

# Compute EMAs
short_term_ema = compute_ema(losses, beta_short)
long_term_ema = compute_ema(losses, beta_long)
plt.scatter(np.arange(start_idx, short_term_ema.shape[0]), (short_term_ema[start_idx:]),s=2, label="Short Term Loss")
plt.yscale("log")
plt.legend()
plt.show()
plt.scatter(np.arange(start_idx, long_term_ema.shape[0]), (long_term_ema[start_idx:]),s=2, label="Long Term Loss")
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
Ndiff = config.max_diff_steps
if "PMS" in config.scoreNet_trained_path:
    type = "PMS"
elif "PM" in config.scoreNet_trained_path:
    type = "PM"
else:
    type = ""

if "fSin" in config.data_path:
    file_path = f"/Users/marcos/GitHubRepos/FractionalBrownianMotion/experiments/results/TS{type}_LSTM_fSin_DriftEvalExp_{Nepoch}Nep_{config.loss_factor}LFactor_10MeanRev_{Ndiff}DiffSteps_"
elif "fQuadSinHF" in config.data_path:
    file_path = (
            project_config.ROOT_DIR + f"experiments/results/TS{type}_LSTM_fQuadSinHF_DriftEvalExp_{Nepoch}Nep_{config.loss_factor}LFactor_{config.t0}t0_{config.deltaT:.3e}dT_{config.quad_coeff}a_{config.sin_coeff}b_{config.sin_space_scale}c_{config.deltaT:.3e}dT_{config.beta_max:.1e}betaMax_").replace(
        ".", "")
    print(file_path)
elif "fBiPotSmall" in config.data_path:
    file_path = (
            project_config.ROOT_DIR + f"experiments/results/TS{type}_LSTM_fBiPotSmall_DriftEvalExp_{Nepoch}Nep_{config.loss_factor}LFactor_{config.quartic_coeff}a_{config.quad_coeff}b_{config.const}c_{config.max_diff_steps}DiffSteps_").replace(
        ".", "")
elif "fBiPot" in config.data_path:
    file_path = (
            project_config.ROOT_DIR + f"experiments/results/TS{type}_LSTM_ST_fBiPot_DriftEvalExp_{Nepoch}Nep_{config.loss_factor}LFactor_{config.t0}t0_{config.deltaT:.3e}dT_{config.quartic_coeff}a_{config.quad_coeff}b_{config.const}c_{config.deltaT:.3e}dT_{config.beta_max:.1e}betaMax_").replace(
        ".", "")
muhats = torch.Tensor(np.load(file_path+"muhats.npy"))

In [ ]:
Xshape = muhats.shape[0]
if "fQuadSinHF" in config.data_path:
    if config.deltaT > 1/(32*256):
        Xs = torch.linspace(-1.2, 1.2, steps=Xshape)
    else:
        Xs = torch.linspace(-.4, .4, steps=Xshape)
    ts_type = "fQuadSinHF"
elif "fSin" in config.data_path:
    Xs = torch.Tensor(np.linspace(-3,3,Xshape))#np.load(file_path+"numpyXs.npy"))
    ts_type = "fSin"
elif "fBiPotSmall" in config.data_path:
    Xs = torch.Tensor(np.linspace(-1.5,1.5,Xshape))#np.load(file_path+"numpyXs.npy"))
    ts_type="fBiPotSmall"
elif "fBiPot" in config.data_path:
    Xs = torch.Tensor(np.linspace(-1.5,1.5,Xshape))#np.load(file_path+"numpyXs.npy"))
    ts_type="fBiPot"
if type == "":
    type="Standard"
print(type, ts_type, Ndiff, muhats.shape)

In [ ]:
def plot_drift_estimator(mean, stds, numpy_Xs, type, true_drift, ts_type,ts_type_num, Nepoch,toSave:bool = True):
    fig, ax = plt.subplots(figsize=(14,9))
    rmse = rmse_ignore_nans(true_drift, mean).astype(np.float64)#np.power(np.mean(np.power(true_drift - mean, 2)), 0.5)
    ax.scatter(numpy_Xs, true_drift, color="red", label="True Drift")

    #ax.errorbar(numpy_Xs, mean, fmt="o",yerr=2*stds, label="Estimated Drift")
    plt.scatter(numpy_Xs, mean, label="Estimated Drift", color="blue")
    ax.set_title(rf"RMSE {round(rmse,3)} for LSTM Score Estimator", fontsize=40)
    ax.tick_params(labelsize=38)
    ax.set_xlabel("State $X$", fontsize=38)
    ax.set_ylabel("Drift Value", fontsize=38)
    ax.legend(fontsize=24)
    plt.tight_layout()
    if toSave:
        plt.savefig(f"/Users/marcos/Library/CloudStorage/OneDrive-ImperialCollegeLondon/StatML_CDT/Year2/DiffusionModelPresentationImages/{ts_type}{ts_type_num}_{type}_LSTM_{Nepoch}Nep.png",  bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
print(type,muhats.shape)
assert (config.max_diff_steps == 10000)
if "fQuadSin" in config.data_path:
    true_drifts = -2.*config.quad_coeff * Xs.unsqueeze(-1) + config.sin_coeff * config.sin_space_scale*np.sin(config.sin_space_scale*Xs).unsqueeze(-1)
    print(config.quad_coeff, config.sin_coeff, config.sin_space_scale)
elif "fSin" in config.data_path:
    true_drifts = config.mean_rev*np.sin(1*Xs).unsqueeze(-1)
elif "BiPot" in config.data_path:
    true_drifts = -(4.*config.quartic_coeff * np.power(Xs, 3) + 2.*config.quad_coeff * Xs + config.const).unsqueeze(-1)
    #true_drifts = true_drifts/(1.+config.deltaT*torch.abs(true_drifts))

In [ ]:
"""
diff_mses = (true_drifts - muhats.mean(dim=(0,-1)).numpy()).permute(1,0).pow(2).sum(dim=-1) / (muhats.shape[0])
diff_rmses = diff_mses.pow(0.5)
fig, ax = plt.subplots(figsize=(14, 9))
ax.plot(np.linspace(config.sample_eps, config.end_diff_time, muhats.shape[1]), diff_rmses)
ax.set_title(r"Drift RMSE Against Diffusion Time $\tau_{s}$", fontsize=40)
ax.tick_params(labelsize=38)
plt.yscale("log")
ax.set_xlabel(r"Diffusion Time $\tau_{s}$", fontsize=48)
ax.set_ylabel(r"RMSE", fontsize=38)
#plt.savefig(f"/Users/marcos/Library/CloudStorage/OneDrive-ImperialCollegeLondon/StatML_CDT/Year2/DiffusionModelPresentationImages/{ts_type}_{type}_RMSEs.png",  bbox_inches='tight')"""

In [ ]:
mu_hats = muhats[:, -1, :].reshape(muhats.shape[0], muhats.shape[-1]*1).mean(dim=-1).numpy()
print(np.mean(mu_hats), np.std(mu_hats))
stds = muhats[:, -1, :].reshape(muhats.shape[0], muhats.shape[-1]*1).std(dim=-1).numpy()
plot_drift_estimator(mean=mu_hats, stds=stds, numpy_Xs=Xs.numpy(), type=type, toSave=toSave, true_drift=true_drifts.numpy(), ts_type=ts_type, ts_type_num=ts_type_num, Nepoch=Nepoch)
print(np.mean(stds))
del muhats, mu_hats, stds, true_drifts